# Imports & globals

In [5]:
import os
import torch
from torch.utils.data import TensorDataset, DataLoader
from analysis_steps import *

In [2]:
DEVICE = "cuda:0"
CODE_DIR = "/home/cbarkhof/fall-2021"

ANALYSIS_DIR = f"{CODE_DIR}/analysis/analysis-files"
CHECKPOINT_DIR = f"{CODE_DIR}/run_files/checkpoints"

SAMPLE_FILE = f"generative-samples.pt"
CONDITIONAL_SAMPLE_FILE = f"generative-conditional-samples.pt"

# Load run overview from W&B data base

In [8]:
prefixes = ["(29-nov ptb-beta-vae)", "(29-nov yahoo-beta-vae)", "(29-nov drop yahoo-beta-vae)"]
overview_df = make_run_overview_df(prefixes, add_data_group=False)
print(len(overview_df))
overview_df

36


,objective,dataset,l_rate,dropout,beta_beta,free_bits,mdr_value,l_mmd,decoder,run_name
yahoo_answer | drop-0.25 | BETA-VAE beta 0.5 dec: Strong roBERTa,BETA-VAE,yahoo_answer,0,0.250000,0.500000,0,0,0,strong_distil_roberta_decoder,(29-nov drop yahoo-beta-vae) B-VAE[b=0.5] | ST...
yahoo_answer | drop-1 | BETA-VAE beta 0.1 dec: Strong roBERTa,BETA-VAE,yahoo_answer,0,1,0.100000,0,0,0,strong_distil_roberta_decoder,(29-nov drop yahoo-beta-vae) B-VAE[b=0.1] | ST...
yahoo_answer | drop-0.75 | BETA-VAE beta 0.1 dec: Strong roBERTa,BETA-VAE,yahoo_answer,0,0.750000,0.100000,0,0,0,strong_distil_roberta_decoder,(29-nov drop yahoo-beta-vae) B-VAE[b=0.1] | ST...
yahoo_answer | drop-0.5 | BETA-VAE beta 0.1 dec: Strong roBERTa,BETA-VAE,yahoo_answer,0,0.500000,0.100000,0,0,0,strong_distil_roberta_decoder,(29-nov drop yahoo-beta-vae) B-VAE[b=0.1] | ST...
yahoo_answer | drop-0.25 | BETA-VAE beta 0.1 dec: Strong roBERTa,BETA-VAE,yahoo_answer,0,0.250000,0.100000,0,0,0,strong_distil_roberta_decoder,(29-nov drop yahoo-beta-vae) B-VAE[b=0.1] | ST...
yahoo_answer | drop-1 | BETA-VAE beta 0 dec: Strong roBERTa,BETA-VAE,yahoo_answer,0,1,0,0,0,0,strong_distil_roberta_decoder,(29-nov drop yahoo-beta-vae) B-VAE[b=0.0] | ST...
yahoo_answer | drop-0.75 | BETA-VAE beta 0 dec: Strong roBERTa,BETA-VAE,yahoo_answer,0,0.750000,0,0,0,0,strong_distil_roberta_decoder,(29-nov drop yahoo-beta-vae) B-VAE[b=0.0] | ST...
yahoo_answer | drop-0.5 | BETA-VAE beta 0 dec: Strong roBERTa,BETA-VAE,yahoo_answer,0,0.500000,0,0,0,0,strong_distil_roberta_decoder,(29-nov drop yahoo-beta-vae) B-VAE[b=0.0] | ST...
yahoo_answer | drop-0.25 | BETA-VAE beta 0 dec: Strong roBERTa,BETA-VAE,yahoo_answer,0,0.250000,0,0,0,0,strong_distil_roberta_decoder,(29-nov drop yahoo-beta-vae) B-VAE[b=0.0] | ST...
yahoo_answer | drop-1 | BETA-VAE beta 1 dec: Strong roBERTa,BETA-VAE,yahoo_answer,0,1,1,0,0,0,strong_distil_roberta_decoder,(29-nov drop yahoo-beta-vae) B-VAE[b=1.0] | ST...


In [7]:
df_missing = overview_of_missing_analysis(overview_df)
df_missing.sum()

dir                        0
surprisal_data            36
surprisal_samples          9
mmd_redo                   0
surprisal_cond_samples     8
samples                    0
cond_samples               0
test_valid_eval            0
dtype: int64

# Collect conditional and unconditional samples

```
conditional_sample_file = f"{ANALYSIS_DIR}/{run_name}/{CONDITIONAL_SAMPLE_FILE}"
sample_file = f"{ANALYSIS_DIR}/{run_name}/{SAMPLE_FILE}"

samples = dict(z=[], x=[], text=[])
conditional_sample_d = {phase: dict(z=[], mean=[], scale=[], 
                                    cond_sample_x=[], condtional_sample_text=[],
                                    original_input_text=[], original_input_ids=[], original_attention_mask=[]) 
                        for phase in phases}
```

In [4]:
from transformers import RobertaTokenizerFast
import torch

n_sample_batches = 24
sample_batch_size = 128
print(f"Running sampling process, with {n_sample_batches*sample_batch_size} (conditional) samples for each checkpoint.")

tok = RobertaTokenizerFast.from_pretrained("distilroberta-base")

with torch.no_grad():
    collect_samples(overview_df, 
                    tok=tok,
                    device=DEVICE, 
                    include_train=True, 
                    n_sample_batches=n_sample_batches, 
                    sample_batch_size=sample_batch_size, 
                    reverse=False)


Running sampling process, with 3072 (conditional) samples for each checkpoint.
Loading yahoo_answer
Is file!
train 140000
validation 6000
test 6000
Loading ptb
Is file!
train 42068
validation 3370
test 3761
0 (29-nov drop yahoo-beta-vae) B-VAE[b=0.5] | STRONG | Nz=128 | drop=0.25 yahoo_answer
Done this run already, continue
1 (29-nov drop yahoo-beta-vae) B-VAE[b=0.1] | STRONG | Nz=128 | drop=1.0 yahoo_answer
Done this run already, continue
2 (29-nov drop yahoo-beta-vae) B-VAE[b=0.1] | STRONG | Nz=128 | drop=0.75 yahoo_answer
Done this run already, continue
3 (29-nov drop yahoo-beta-vae) B-VAE[b=0.1] | STRONG | Nz=128 | drop=0.5 yahoo_answer
Done this run already, continue
4 (29-nov drop yahoo-beta-vae) B-VAE[b=0.1] | STRONG | Nz=128 | drop=0.25 yahoo_answer
Done this run already, continue
5 (29-nov drop yahoo-beta-vae) B-VAE[b=0.0] | STRONG | Nz=128 | drop=1.0 yahoo_answer
Done this run already, continue
6 (29-nov drop yahoo-beta-vae) B-VAE[b=0.0] | STRONG | Nz=128 | drop=0.75 yahoo_an

# Evaluate validation and test global statistics

In [5]:
simple_evaluate_valid_test(overview_df, 
                           device=DEVICE, 
                           batch_size=128, 
                           num_workers=3)


First re-computing MMD with BASE_ALPHA = 0.001
yahoo_answer | drop-0.25 | BETA-VAE beta 0.5 dec: Strong roBERTa
yahoo_answer | drop-1 | BETA-VAE beta 0.1 dec: Strong roBERTa
yahoo_answer | drop-0.75 | BETA-VAE beta 0.1 dec: Strong roBERTa
yahoo_answer | drop-0.5 | BETA-VAE beta 0.1 dec: Strong roBERTa
yahoo_answer | drop-0.25 | BETA-VAE beta 0.1 dec: Strong roBERTa
yahoo_answer | drop-1 | BETA-VAE beta 0 dec: Strong roBERTa
yahoo_answer | drop-0.75 | BETA-VAE beta 0 dec: Strong roBERTa
yahoo_answer | drop-0.5 | BETA-VAE beta 0 dec: Strong roBERTa
yahoo_answer | drop-0.25 | BETA-VAE beta 0 dec: Strong roBERTa
yahoo_answer | drop-1 | BETA-VAE beta 1 dec: Strong roBERTa
yahoo_answer | drop-0.75 | BETA-VAE beta 1 dec: Strong roBERTa
yahoo_answer | drop-0.5 | BETA-VAE beta 1 dec: Strong roBERTa
yahoo_answer | drop-0.25 | BETA-VAE beta 1 dec: Strong roBERTa
yahoo_answer | drop-1 | BETA-VAE beta 0.5 dec: Strong roBERTa
yahoo_answer | drop-0.75 | BETA-VAE beta 0.5 dec: Strong roBERTa
yahoo_ans

# Collect surprisal statistics

In [ ]:
collect_data_surprisal_stats(overview_df, batch_size_surprisal=4, iw_ll_n_samples=40, num_workers=3,
                             max_data_samples=2000, device=DEVICE)

Loading yahoo_answer
Is file!
train 140000
validation 6000
test 6000
Loading ptb
Is file!
train 42068
validation 3370
test 3761
yahoo_answer | drop-0.25 | BETA-VAE beta 0.5 dec: Strong roBERTa (29-nov drop yahoo-beta-vae) B-VAE[b=0.5] | STRONG | Nz=128 | drop=0.25


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializin

beta_kl: 5.34 | total_loss: 161.39 | mmd: 0.00 | elbo: -166.73 | distortion: 156.05 | kl_prior_post: 10.69 | mean_mean: -0.00 | std_across_x_mean: 0.31 | std_across_z_mean: 0.35 | mean_scale: 0.92 | std_across_x_scale: 0.04 | std_across_z_scale: 0.10 | 
